In [32]:
import pandas as pd
import sqlite3

path = "../findajob.db"
conn =  sqlite3.connect(path)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

tables = cursor.fetchall()

# print("overview of tables:")
# for table in tables:
#     print(table)
    
job_types = "SELECT * FROM job_types ORDER BY type_name"
df_job_types = pd.read_sql(job_types, conn)

result_types = "SELECT * FROM results"
df_result_types = pd.read_sql(result_types, conn)

# pending jobs
pending_jobs = "SELECT * FROM openings WHERE result = 1"
df_pending_jpbs = pd.read_sql(pending_jobs, conn)

# ghosted jobs
ghosted_jobs = "SELECT * FROM openings WHERE result = 5"
df_ghosted_jpbs = pd.read_sql(ghosted_jobs, conn)

# rejected jobs
rejected_jobs = "SELECT * FROM openings WHERE result = 3"
df_rejected_jpbs = pd.read_sql(rejected_jobs, conn)

# interviewed jobs
interview_jobs = "SELECT * FROM openings WHERE result = 2"
df_interview_jpbs = pd.read_sql(interview_jobs, conn)

count_pending_jobs = len(df_pending_jpbs)
print(f"I am wating for {count_pending_jobs} companies to respond")

count_ghosted_jpbs = len(df_ghosted_jpbs)
print(f"I was ghosted by {count_ghosted_jpbs} companies")

count_rejected_jpbs = len(df_rejected_jpbs)
print(f"I was rejected by {count_rejected_jpbs} companies")

count_interviewed_jpbs = len(df_interview_jpbs)
print(f"I had {count_interviewed_jpbs} interviews")

count_applications = sum([count_pending_jobs, count_ghosted_jpbs, count_rejected_jpbs, count_interviewed_jpbs])
print(f"I applied for {count_applications} jobs in total")
print("-------------")
ghost_rate = len(df_ghosted_jpbs)/count_applications*100
print(f"My ghost rate is: {round(ghost_rate)}%")

interview_rate = len(df_interview_jpbs)/count_applications*100
print(f"My interview rate is: {round(interview_rate)}%")
fail_rate = count_ghosted_jpbs+count_rejected_jpbs/count_applications
print(f"My fail rate is: {round(fail_rate)}%")
print("-------------")



print(df_result_types)


conn.close()

I am wating for 27 companies to respond
I was ghosted by 88 companies
I was rejected by 48 companies
I had 3 interviews
I applied for 166 jobs in total
-------------
My ghost rate is: 53%
My interview rate is: 2%
My fail rate is: 88%
-------------
   id    result_name
0   1        pending
1   2      interview
2   3       rejected
3   4          hired
4   5        ghosted
5   6  want to apply
